# Orders

Gran parte de nuestra tarea es crear "Features", variables en los datos que representan un aspecto del fenómeno que estudiamos y nos ayudan a explicarlo.

**Ejercicio**

Vamos a investigar la relación entre orders y el score.

La idea es crear un dataframe con las siguientes features:

| feature_name 	                | type 	| description 	                                                                                                                                         |
|:------------------------------|:---:	|:------------------------------------------------------------------------------------------------------------------------------------------------------|
| `order_id` 	                  | str 	| the id of the order 	                                                                                                                                 |
| `tiempo_de_espera` 	          | float 	| numero de días entre order_date y delivered_date 	                                                                                                    |
| `tiempo_de_espera_previsto` 	 | float 	| numero de días entre order_date y estimated_delivery_date 	                                                                                           |
| `real_vas_esperado` 	         | float 	| si la fecha de entrega real es posterior a la fecha de entrega estimada, devuelve el número de días entre las dos fechas; de lo contrario, devuelve 0 |
| `order_status` 	              | str 	| statusde la orden	                                                                                                                                    |
| `es_cinco_estrellas` 	        | int 	| 1 si tuvo cinco estrellas en la  review, 0 caso contrario 	                                                                                           |
| `es_una_estrella` 	           | int 	| 1 si tuvo una estrella, 0 caso contrario 	                                                                                                            |
| `review_score` 	              | int 	| from 1 to 5 	                                                                                                                                         |
| `numero_de_productos` 	       | int 	| numero de productos que contiene una orden	                                                                                                           |
| `numero_de_vendedores` 	      | int 	| numero de vendedores unicos                                                                                                                           |
| `precio` 	                    | float 	| total del precio pagado por el cliente                                                                                                                |
| `valor_transporte` 	          | float 	| el valor del freight pagado por el cliente 	                                                                                                          |
| `distancia_al_cliente` 	      | float 	| distancia en km entre cliente y vendedor 	                                                                                                  |
  
Queremos filtrar las ordenes no entregadas

## Metodología:

Vamos a crear dos notebooks una de testeo y otra para importar nuestro paquete preprocess.orders_data y de ahí ir elavorando los métodos.

1er Método: Load_data: Trae el dataset sin nulos de una ubicación ya prefijada.

###  Buenas Pracitcas


Las notebooks se pueden hacer muy largas y desordenadas.
- Codifique su lógica para que siempre se pueda ejecutar de arriba a abajo sin fallar (Celda --> Ejecutar todo)
- Nombre sus variables con lo que significa y para que otra persona pueda entenderlo.
- Use nombres como `tmp` o `_` para pasos intermedios si sabe que no va a usarlos mucho tiempo
- Borre su código y combine celdas cuando sea relevante para minimizar el tamaño del Notebook (`Shift-M`)
- Oculte la salida de su celda si ya no necesita verla (haga doble clic en la sección roja `Out[]:` a la izquierda de su celda).
- Utilice la Structure de pycharm o Juypter lab para organizarse
- Shortcuts
    - `a` para insertar una celda arriba
    - `b` para insertar una celda debajo
    - `dd` para borrar una celda
    - `esc` y `flechas` para moverse entre celdas
    - `Shift-Enter` para ejecutar la celda y mover el foco a la siguiente
    - `Shift + Tab` cuando entre paréntesis de método, p. ¡`groupby()` para obtener los documentos! Repita varias veces para abrirlo permanentemente

In [11]:
%load_ext autoreload
%autoreload 2

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Import olist data
from olist.load_data import load_all_data
data = load_all_data("data/")

In [3]:
data.keys()

dict_keys(['customers', 'geolocation', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'product_category_name_translation'])

In [56]:
orders = data['orders'].copy() # buena practica para estar seguro que no modificas tu dataset original

## 1. Crear orders.py

### a) `tiempo_de_espera`
    Armar un dataframe que tenga
           order_id, tiempo_de_espera, tiempo_de_espera_previsto, real_vs_esperado, status_de_la_orden


In [57]:
orders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

Filtro solo las columnas que su status sea delivered

In [58]:
orders = orders[orders["order_status"] == "delivered"]

Paso todas las columnas que deberían ser fecha a su formato correspondiente

In [59]:
date_columns = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
                'order_delivered_customer_date', 'order_estimated_delivery_date']

# Convertir las columnas a objetos datetime
for column in date_columns:
    orders[column] = pd.to_datetime(orders[column])

In [60]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


Filtro solo aquellas columnas que voy a necesitar

In [61]:
orders = orders[['order_id', 'order_status', 'order_purchase_timestamp',
                'order_delivered_customer_date', 'order_estimated_delivery_date']]

Creo las columnas tiempo_de_espera y tiempo_de_espera_previsto como diferencia entre fechas, asignando también el tipo de dato que se desea

In [62]:
orders['tiempo_de_espera'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days.astype(float)

In [63]:
orders['tiempo_de_espera_previsto'] = (orders['order_estimated_delivery_date'] - orders['order_purchase_timestamp']).dt.days.astype(float)

Creo la columna real_vs_esperado y despues le asigno un 0 a todas las columnas que no sean numeros positivos

In [64]:
orders["real_vs_esperado"] = (orders['tiempo_de_espera'] - orders['tiempo_de_espera_previsto'])

In [65]:
orders['real_vs_esperado'] = orders['real_vs_esperado'].apply(lambda x: x if x > 0 else 0)

Elimino columnas innecesarias

In [66]:
orders.drop('order_purchase_timestamp', axis=1, inplace=True)
orders.drop('order_delivered_customer_date', axis=1, inplace=True)
orders.drop('order_estimated_delivery_date', axis=1, inplace=True)

Renombro para que quede con el formato adecuado

In [67]:
orders.rename(columns={'order_status': 'status_de_la_orden'}, inplace=True)

In [71]:
orders.dtypes

order_id                      object
status_de_la_orden            object
tiempo_de_espera             float64
tiempo_de_espera_previsto    float64
real_vs_esperado             float64
dtype: object

In [69]:
orders.sample(5)

,order_id,status_de_la_orden,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado
98486,092de8ef0dad90862e5c0dfb15a10790,delivered,9.0,28.0,0.0
57706,bee4fbc438d2d48527e812cb6a1c8166,delivered,43.0,22.0,21.0
48008,84eb86f5f4ba017cd1644c27f6140424,delivered,23.0,31.0,0.0
41663,b624512c534133dd5a23c9a1acb8e2b0,delivered,7.0,16.0,0.0
99315,331027db835328af30e85ce6c7b55655,delivered,14.0,28.0,0.0


# Pasos a seguir

> 1. Explora el dataframe `orders`
2. Filtra por `delivered orders`
3. Manipulá correctamente `datetime` (recordá el examen)
    - Repasá[`datetime`](https://docs.python.org/3/library/datetime.html)
    - Métodos para pasar "string" type a "pandas.datetime'  [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
4. Crea la columna `tiempo_de_espera`
5. Crea la columna  `tiempo_de_espera_previsto`
6. Crea la columna  `real_vs_esperado`
7. Chequea el nuevo dataset
8. Una vez terminado el código pasalo a tu librería `order.py`

Tanto para `tiempo_de_espera` como para `tiempo_de_espera_previsto`, debe restar las fechas/marcas de tiempo relevantes para obtener la diferencia horaria entre los objetos `pandas.datetime`. Luego, puede usar [`datetime.timedelta()`](https://docs.python.org/3/library/datetime.html#timedelta-objects) o [`np.timedelta64()`](https ://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-and-timedelta-arithmetic) para averiguar cuántos días representa esa resta.


Chequeo que me dije importar el dataset

In [97]:
from olist.order import orders

In [98]:
orders.head(2)

,order_id,status_de_la_orden,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,8.0,15.0,0.0
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,13.0,19.0,0.0


### Crear una columna
Es lo mismo utilizar .loc para asignar una columna nueva, que utilizar .assign o simplemente df['columna'] = operación

##  Compute just the number of days in each time_delta 
import datetime
one_day_delta = datetime.timedelta(days=1) # lleva el time delta a un día
one_day_delta = np.timedelta64(24, 'h') # también se puede hacer de esta manera


* \ se puede usar para ahorrar espacio
orders.loc[:,'tiempo_de_espera'] = \
    (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']) / one_day_delta

(orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.da


# pasarlo a modulo

Crear la funcion tiempo_de_espera e implementarla

to write python code


def tiempo_de_espera(orders, is_delivered=True):
    # filtrar por entregados y crea la varialbe tiempo de espera
    if is_delivered:
        orders = orders.query("order_status=='delivered'").copy()
    # compute wait time
    orders.loc[:, 'tiempo_de_espera'] = \
        (orders['order_delivered_customer_date'] -
         orders['order_purchase_timestamp']) / np.timedelta64(24, 'h')
    return orders

def

### b) `puntaje_de_compra`
     retorna un dataframe con
        order_id, es_cinco_estrellas, es_una_estrella, review_score

recordar del examen `DataFrame.apply()` o tmb se puede usar`Series.map()`


👉 We load the `reviews` for you

In [89]:
reviews = data['order_reviews'].copy()

reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [90]:
reviews.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

Me quedo solo con las columnas que voy a necesitar

In [91]:
reviews = reviews[["order_id", "review_score"]]

Creo las nuevas columnas

In [92]:
reviews['es_cinco_estrellas'] = reviews['review_score'].apply(lambda x: 1 if x == 5 else 0)
reviews['es_una_estrella'] = reviews['review_score'].apply(lambda x: 1 if x == 1 else 0)

Chequeo que funcionen bien mediante un sample

In [95]:
reviews.sample(4)

,order_id,review_score,es_cinco_estrellas,es_una_estrella
14240,5ba2971983dbd7066539c01bcaad029d,5,1,0
19247,30a0d918b3ba882cd4f3557a0ac1182c,4,0,0
17556,7e60cf16935261e615cfe1e000aa05e7,4,0,0
91575,2508a4e7eaf40f21355892834c2eb54d,1,0,1


### c) `numero_de_productos`:
     retorna un dataframe con
        order_id, numero_de_productos (numero_de_productos_por_orden)

In [123]:
order_items = data['order_items'].copy()

In [124]:
order_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


Agrupo y cuento cuantas veces se repite el order_id para saber la cantidad de productos que contiene

In [125]:

order_items = order_items.groupby('order_id').size().reset_index(name='numero_de_productos')

In [126]:
order_items.dtypes

order_id               object
numero_de_productos     int64
dtype: object

### d) `numero_de_vendedores`:
     Retorna un dataframe con
        order_id, numero_de_vendedores (total de vendedores UNICOS !!! )

HINT!:
`pd.Series.nunique()`


In [129]:
data.keys()

dict_keys(['customers', 'geolocation', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'product_category_name_translation'])

In [138]:
order_items = data['order_items'].copy()

In [140]:
order_items.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

Agrupo por order_id y cuento los valores unicos de vendedores por cada orden

In [141]:
numero_de_vendedores = order_items.groupby('order_id')['seller_id'].nunique().reset_index()

Renombro la columna para que el título sea más representativo

In [144]:
numero_de_vendedores = numero_de_vendedores.rename(columns={'seller_id': 'numero_de_vendedores'})

### e) `calcular_precio_y_transporte`
     Returns a DataFrame with:
        order_id, precio, valor_transporte

recordar el agg en el groupby

In [145]:
data.keys()

dict_keys(['customers', 'geolocation', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'product_category_name_translation'])

In [165]:
order_items = data['order_items'].copy()

Sumo cada línea de la orden para obtener el precio total de la orden y el precio total del transporte de la orden

In [166]:

calcular_precio_y_transporte = order_items.groupby('order_id')[['price', 'freight_value']].sum().reset_index()

In [167]:
calcular_precio_y_transporte

,order_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,299.99,43.41
98662,fffcd46ef2263f404302a634eb57f7eb,350.00,36.53
98663,fffce4705a9662cd70adb13d4a31832d,99.90,16.95
98664,fffe18544ffabc95dfada21779c9644f,55.99,8.72


### e) `distancia_al_cliente`

    Retorna un dataframe con
        order_id, distancia_al_cliente (distancia en kilometros)

usar`haversine_distance`

Veo que columnas tiene cada csv para saber cuales voy a necesitar

In [161]:
for nombre in data.keys():
    print(nombre)
    nombre = data[nombre].copy()
    print(nombre.columns)

customers
Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')
geolocation
Index(['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'geolocation_city', 'geolocation_state'],
      dtype='object')
orders
Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
order_items
Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
order_payments
Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')
order_reviews
Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'revie

In [229]:
from math import radians, sin, cos, asin, sqrt

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Computa distancia entre dos pares (lat, lng)
    Ver - (https://en.wikipedia.org/wiki/Haversine_formula)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))


In [230]:
sellers = data['sellers'].copy()
geolocation = data['geolocation'].copy()

In [231]:
sellers.columns

Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')

In [232]:
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


Agrupo por zipcode y traigo el primer valor de lat y lon ya que un mismo zipcode puede tener muchas latitudes y longitudes y necesitamos solamente una por zipcode

In [233]:
geo = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].first().reset_index()

In [234]:
geo.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,1001,-23.549292,-46.633559
1,1002,-23.548318,-46.635421
2,1003,-23.549032,-46.635313
3,1004,-23.550116,-46.635122
4,1005,-23.549819,-46.635606


Renombro la columna para poder mergearla con el dataset de sellers

In [235]:
geo_sellers = geo.rename(columns={'geolocation_zip_code_prefix': 'seller_zip_code_prefix'})

Me quedo con las columnas que necesito

In [236]:
sellers = sellers[['seller_id', 'seller_zip_code_prefix']]

Hago el merge left para que todos los sellers tengan su lat y lon y que no traiga aquellos registros que no tienen coincidencias en el dataset de sellers

In [237]:
sellers_loc = pd.merge(sellers, geo_sellers, on='seller_zip_code_prefix', how='left')

Renombro las columnas para que, a la hora de traer las lat y lon de los compradores no se pisen ni se confundan

In [238]:
sellers_loc = sellers_loc.rename(columns={'geolocation_lat': 'lat_seller', "geolocation_lng" : "lon_seller"})

In [239]:
sellers_loc

,seller_id,seller_zip_code_prefix,lat_seller,lon_seller
0,3442f8959a84dea7ee197c632cb2df15,13023,-22.898536,-47.063125
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,-22.382941,-46.946641
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,-22.910641,-43.176510
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,-23.657250,-46.610759
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,-22.971648,-46.533618
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,-23.456431,-51.866369
3091,f8201cab383e484733266d1906e2fdfa,88137,-27.623801,-48.674286
3092,74871d19219c7d518d0090283e03c137,4650,-23.659845,-46.677882
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,-31.744231,-52.328761


Hago lo mismo pero con los clientes, mismo proceso

In [240]:
customers = data['customers'].copy()

geo_customers = geo.rename(columns={'geolocation_zip_code_prefix': 'customer_zip_code_prefix'})
customers = customers[['customer_id', 'customer_zip_code_prefix']]

customers_loc = pd.merge(customers, geo_customers, on='customer_zip_code_prefix', how='left')
customers_loc = customers_loc.rename(columns={'geolocation_lat': 'lat_customer', "geolocation_lng" : "lon_customer"})

In [241]:
customers_loc

,customer_id,customer_zip_code_prefix,lat_customer,lon_customer
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,-20.509897,-47.397866
1,18955e83d337fd6b2def6b18a428ac77,9790,-23.726853,-46.545746
2,4e7b3e00288586ebd08712fdd0374a03,1151,-23.527788,-46.660310
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,-23.496930,-46.185352
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,-22.987222,-47.151073
...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,3937,-23.587901,-46.501830
99437,e7b71a9017aa05c9a7fd292d714858e8,6764,-23.612294,-46.765787
99438,5e28dfe12db7fb50a4b2f691faecea5e,60115,-3.744128,-38.510859
99439,56b18e2166679b8a959d72dd06da27f9,92120,-29.956391,-51.167614


La union entre seller y customers se da así:

sellers <-seller_id-> order_item <-order_id-> orders <-customer_id->customers

Por lo tanto voy a traer los dos dataset que las conectan sellers y customer y me quedaré con solo las columnas necesarias para los merges

In [242]:
order_items = data['order_items'].copy()
order_items.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

In [243]:
order_items = order_items[['order_id','order_item_id', 'seller_id']]

In [244]:
order_seller = pd.merge(order_items, sellers_loc, on='seller_id', how='left')

In [245]:
orders = data['orders'].copy()
orders = orders[['order_id', 'customer_id']]

Uno con el de sellers

In [276]:
distancias = pd.merge(order_seller, orders, on='order_id', how='left')

In [277]:
distancias

,order_id,order_item_id,seller_id,seller_zip_code_prefix,lat_seller,lon_seller,customer_id
0,00010242fe8c5a6d1ba2dd792cb16214,1,48436dade18ac8b2bce089ec2a041202,27277,-22.498183,-44.123614,3ce436f183e68e07877b285a838db11a
1,00018f77f2f0320c557190d7a144bdd3,1,dd7ddc04e1b6c2c614352b383efe2d36,3471,-23.566258,-46.518417,f6dd3ec061db4e3987629fe6b26e5cce
2,000229ec398224ef6ca0657da4fc703e,1,5b51032eddd242adc84c38acab88f23d,37564,-22.264094,-46.158564,6489ae5e4333f3693df5ad4372dab6d3
3,00024acbcdf0a6daa1e931b038114c75,1,9d7a1d34a5052409006425275ba1c2b4,14403,-20.548228,-47.395897,d4eb9395c8c0431ee92fce09860c5a06
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,df560393f3a51e74553ab94004ba5c87,87900,-22.931427,-53.133759,58dbd0b2d70206bf40e62cd34e84d795
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,b8bc237ba3788b23da09c0f1f3a3288c,88303,-26.912429,-48.677381,b51593916b4b8e0d6f66f2ae24f2673d
112646,fffcd46ef2263f404302a634eb57f7eb,1,f3c38ab652836d21de61fb8314b69182,1206,-23.535536,-46.643052,84c5d4fbaf120aae381fad077416eaa0
112647,fffce4705a9662cd70adb13d4a31832d,1,c3cfdc648177fdbbbb35635a37472c53,80610,-25.469214,-49.293935,29309aa813182aaddc9b259e31b870e6
112648,fffe18544ffabc95dfada21779c9644f,1,2b3e4a2a3ea8e01938cabda2a3e5cc79,4733,-23.636657,-46.694780,b5e6afd5a41800fdf401e0272ca74655


Uno con el de customers

In [278]:
distancias = pd.merge(distancias, customers_loc, on='customer_id', how='left')

In [279]:
distancias

,order_id,order_item_id,seller_id,seller_zip_code_prefix,lat_seller,lon_seller,customer_id,customer_zip_code_prefix,lat_customer,lon_customer
0,00010242fe8c5a6d1ba2dd792cb16214,1,48436dade18ac8b2bce089ec2a041202,27277,-22.498183,-44.123614,3ce436f183e68e07877b285a838db11a,28013,-21.758076,-41.312633
1,00018f77f2f0320c557190d7a144bdd3,1,dd7ddc04e1b6c2c614352b383efe2d36,3471,-23.566258,-46.518417,f6dd3ec061db4e3987629fe6b26e5cce,15775,-20.212393,-50.941471
2,000229ec398224ef6ca0657da4fc703e,1,5b51032eddd242adc84c38acab88f23d,37564,-22.264094,-46.158564,6489ae5e4333f3693df5ad4372dab6d3,35661,-19.860439,-44.597972
3,00024acbcdf0a6daa1e931b038114c75,1,9d7a1d34a5052409006425275ba1c2b4,14403,-20.548228,-47.395897,d4eb9395c8c0431ee92fce09860c5a06,12952,-23.144923,-46.539830
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,df560393f3a51e74553ab94004ba5c87,87900,-22.931427,-53.133759,58dbd0b2d70206bf40e62cd34e84d795,13226,-23.249008,-46.824961
...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,b8bc237ba3788b23da09c0f1f3a3288c,88303,-26.912429,-48.677381,b51593916b4b8e0d6f66f2ae24f2673d,65077,-2.490420,-44.303287
112646,fffcd46ef2263f404302a634eb57f7eb,1,f3c38ab652836d21de61fb8314b69182,1206,-23.535536,-46.643052,84c5d4fbaf120aae381fad077416eaa0,81690,-25.601485,-49.317700
112647,fffce4705a9662cd70adb13d4a31832d,1,c3cfdc648177fdbbbb35635a37472c53,80610,-25.469214,-49.293935,29309aa813182aaddc9b259e31b870e6,4039,-23.597813,-46.650272
112648,fffe18544ffabc95dfada21779c9644f,1,2b3e4a2a3ea8e01938cabda2a3e5cc79,4733,-23.636657,-46.694780,b5e6afd5a41800fdf401e0272ca74655,13289,-23.036935,-46.982526


Aplico la función para calcular la distancia entre seller y cliente

In [280]:
distancias['distancia_al_cliente'] = distancias.apply(lambda row: haversine_distance(row['lon_seller'], row['lat_seller'], row['lon_customer'], row['lat_customer']), axis=1)

Agrupo por order_id y caluclo el promedio de la distancia, ya que hasta el momento, cada orden estaba abierta por order_item y una misma orden podía tener más de un producto, por ende más de un vendedor, a distintas distancias

In [281]:
distancias = distancias.groupby("order_id")["distancia_al_cliente"].mean().reset_index()

Elimino los vacios

In [283]:
distancias = distancias.dropna()

In [284]:
distancias.isna().sum()

order_id                0
distancia_al_cliente    0
dtype: int64

In [285]:
distancias

,order_id,distancia_al_cliente
0,00010242fe8c5a6d1ba2dd792cb16214,301.005664
1,00018f77f2f0320c557190d7a144bdd3,589.274140
2,000229ec398224ef6ca0657da4fc703e,312.495046
3,00024acbcdf0a6daa1e931b038114c75,301.951753
4,00042b26cf59d7ce69dfabb4e55b4fd9,646.221788
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,2755.314485
98662,fffcd46ef2263f404302a634eb57f7eb,354.848852
98663,fffce4705a9662cd70adb13d4a31832d,338.827218
98664,fffe18544ffabc95dfada21779c9644f,72.870476


cual es la mediana de la distancia

In [27]:
distancia_a_la_orden['distance_seller_customer'].describe()

count    98177.000000
mean       601.961534
std        596.877838
min          0.000000
25%        186.380313
50%        433.698770
75%        798.435455
max       8736.947618
Name: distance_seller_customer, dtype: float64

# Merger todos los datasets excepto el de distancia

In [297]:
from olist.order import orders, reviews, numero_de_productos, numero_de_vendedores, calcular_precio_y_transporte, distancias

In [298]:
tablon_primario = orders.merge(reviews, on='order_id') \
    .merge(numero_de_productos, on='order_id') \
    .merge(numero_de_vendedores, on='order_id') \
    .merge(calcular_precio_y_transporte, on='order_id')\
     .merge(distancias, on='order_id')
tablon_primario

,order_id,status_de_la_orden,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado,review_score,es_cinco_estrellas,es_una_estrella,numero_de_productos,numero_de_vendedores,price,freight_value,distancia_al_cliente
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,8.0,15.0,0.0,4,0,0,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,13.0,19.0,0.0,4,0,0,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,9.0,26.0,0.0,5,1,0,1,1,159.90,19.22,514.130333
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,13.0,26.0,0.0,5,1,0,1,1,45.00,27.20,1822.800366
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2.0,12.0,0.0,5,1,0,1,1,19.90,8.72,30.174037
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95875,9c5dedf39a927c1b2549525ed64a053c,delivered,8.0,18.0,0.0,5,1,0,1,1,72.00,13.08,69.481037
95876,63943bddc261676b46f01ca7ac2f7bd8,delivered,22.0,23.0,0.0,4,0,0,1,1,174.90,20.10,474.098245
95877,83c1379a015df1e13d02aae0204711ab,delivered,24.0,30.0,0.0,5,1,0,1,1,205.99,65.02,968.051192
95878,11c177c8e97725db2631073c19f07b62,delivered,17.0,37.0,0.0,2,0,0,2,1,359.98,81.18,370.146853


In [30]:
tablon_primario = orders.merge(reviews, on='order_id') \
    .merge(productos, on='order_id') \
    .merge(vendedores, on='order_id') \
    .merge(precio_y_transporte, on='order_id')\
     .merge(distancia_a_la_orden, on='order_id')
tablon_primario

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,tiempo_de_espera,real_vs_esperado,es_cinco_estrellas,es_una_estrella,review_score,number_of_products,vendedores_unicos,precio,transporte,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.436574,0.0,0,0,4,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.782037,0.0,0,0,4,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.394213,0.0,1,0,5,1,1,159.90,19.22,514.130333
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13.208750,0.0,1,0,5,1,1,45.00,27.20,1822.800366
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2.873877,0.0,1,0,5,1,1,19.90,8.72,30.174037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95875,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,8.218009,0.0,1,0,5,1,1,72.00,13.08,69.481037
95876,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,22.193727,0.0,0,0,4,1,1,174.90,20.10,474.098245
95877,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,24.859421,0.0,1,0,5,1,1,205.99,65.02,968.051192
95878,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,17.086424,0.0,0,0,2,2,1,359.98,81.18,370.146853


In [302]:
tablon_primario.to_csv("data/tablon_primario.csv")